In [ ]:
import cv2
import numpy as np
import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
from collections import deque
import math

In [ ]:
image = cv2.imread('sar_1.jpg')
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


In [ ]:
def homo_average(img, mask, point, T):
    avg_val = img[mask > 0].sum() / np.count_nonzero(mask)
    if abs(avg_val - img[point]) <= T:
        return True
    return False


def region_growing(image, seed_point, homo_fun, r, T):
    mask = np.zeros(image.shape, np.uint8)
    mask[seed_point] = 1
    count = 1
    while count > 0:
        count = 0
        local_mask = np.zeros(image.shape, np.uint8)
        for i in range(r, image.shape[0] - r):
            for j in range(r, image.shape[1] - r):
                if mask[i, j] == 0 and mask[i - r:i + r, j - r:j + r].sum() > 0:
                    if homo_fun(image, mask, (i, j), T):
                        local_mask[i, j] = 1
        count = np.count_nonzero(local_mask)
        mask += local_mask
    return mask * 255


seed_point = (180, 160)
r = 2                    
T = 15                   

mask = region_growing(image_gray, seed_point, homo_average, r, T)

plt.subplot(1, 2, 2)
plt.title("Обработанное изображение (газон выделен)")
plt.imshow(mask, cmap="gray")
plt.show()


![Изображение](Figure_1.png)

In [ ]:
img = cv2.imread("sar_1.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def homo_average(img, mask, point, T):
    avg_val = img[mask > 0].sum() / np.count_nonzero(mask)
    if abs(avg_val - img[point]) <= T:
        return True
    return False


def region_growing(image, seed_point, homo_fun, r, T):
    mask = np.zeros(image.shape, np.uint8)
    mask[seed_point] = 1
    count = 1
    while count > 0:
        count = 0
        local_mask = np.zeros(image.shape, np.uint8)
        for i in range(r, image.shape[0] - r):
            for j in range(r, image.shape[1] - r):
                if mask[i, j] == 0 and mask[i - r:i + r, j - r:j + r].sum() > 0:
                    if homo_fun(image, mask, (i, j), T):
                        local_mask[i, j] = 1
        count = np.count_nonzero(local_mask)
        mask += local_mask
    return mask * 255


seed_point = (180, 160)
r = 2
T = 15

mask = region_growing(gray, seed_point, homo_average, r, T)

plt.subplot(1, 2, 2)
plt.title("Обработанное изображение (газон выделен)")
plt.imshow(mask, cmap="gray")
plt.show()

# 2. Реализуйте вычисление критерия однородности, отличного от представленного. Сравните результаты.
# 2. Разрастание региона с медианным критерием
def uniform_median(img, current_mask, pos, threshold):
    if np.count_nonzero(current_mask) == 0:
        return False
    med = np.median(img[current_mask > 0])
    return abs(int(img[pos]) - med) <= threshold

def region_growing(image, seed_point, homo_fun, r, T):
    mask = np.zeros(image.shape, np.uint8)
    mask[seed_point] = 1
    count = 1
    while count > 0:
        count = 0
        local_mask = np.zeros(image.shape, np.uint8)
        for i in range(r, image.shape[0] - r):
            for j in range(r, image.shape[1] - r):
                if mask[i, j] == 0 and mask[i - r:i + r, j - r:j + r].sum() > 0:
                    if homo_fun(image, mask, (i, j), T):
                        local_mask[i, j] = 1
        count = np.count_nonzero(local_mask)
        mask += local_mask
    return mask * 255

seed = (200, 200)
region_mask_median = region_growing(gray, seed, uniform_median, 2, 21)

plt.figure(figsize=(11, 5))
plt.subplot(1, 2, 1)
plt.imshow(mask, cmap='gray')
plt.title("Средний критерий")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(region_mask_median, cmap='gray')
plt.title("Медианный критерий")
plt.axis('off')
plt.show()

![Изображение](Figure_2.png)

In [ ]:
# 3. Сегментация пальм методом Watershed + Distance Transform

palms = cv2.imread('palm_1.jpg')
gray = cv2.cvtColor(palms, cv2.COLOR_BGR2GRAY)

blur = cv2.GaussianBlur(gray, (5, 5), 0)

_, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

if np.mean(thresh) < 127:
    thresh = 255 - thresh

dist = cv2.distanceTransform(thresh, cv2.DIST_L2, 5)

_, peaks = cv2.threshold(dist, 0.37 * dist.max(), 255, 0)
peaks = np.uint8(peaks)

_, markers = cv2.connectedComponents(peaks)
markers = markers + 1

markers_ws = cv2.watershed(palms, markers)

result_mask = np.zeros_like(gray)
result_mask[markers_ws > 1] = 255


fig, ax = plt.subplots(2, 3, figsize=(14, 9))

ax[0, 0].imshow(cv2.cvtColor(palms, cv2.COLOR_BGR2RGB))
ax[0, 0].set_title('1. Исходное изображение')
ax[0, 0].axis('off')

ax[0, 1].imshow(blur, cmap='gray')
ax[0, 1].set_title('2. Размытие (Gaussian Blur)')
ax[0, 1].axis('off')

ax[0, 2].imshow(thresh, cmap='gray')
ax[0, 2].set_title('3. Пороговая бинаризация')
ax[0, 2].axis('off')

ax[1, 0].imshow(dist, cmap='magma')
ax[1, 0].set_title('4. Distance Transform')
ax[1, 0].axis('off')

ax[1, 1].imshow(peaks, cmap='gray')
ax[1, 1].set_title('5. Пики DT (маркеры)')
ax[1, 1].axis('off')

ax[1, 2].imshow(result_mask, cmap='gray')
ax[1, 2].set_title('6. Итоговая маска (пальмы)')
ax[1, 2].axis('off')

plt.tight_layout()
plt.show()

num_labels, _, stats, _ = cv2.connectedComponentsWithStats(result_mask)
num_palms = num_labels - 1
print(f"Примерное количество пальм: {num_palms}")

![Изображение](Figure_3.png)

Примерное количество пальм: 71